# Query rewriting

In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import os
from typing import List, TypedDict
from dotenv import load_dotenv

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from langchain_core.prompts import ChatPromptTemplate

from src import utils, conf

# Params

In [64]:
conf_settings = conf.load(file="settings.yaml")
conf_infra = conf.load(file="infra.yaml")    

LLM_WORKHORSE = conf_settings.llm_workhorse
LLM_FLAGSHIP = conf_settings.llm_flagship
EMBEDDINGS = conf_settings.embeddings
VDB_URL = conf_infra.vdb_url
INDEX_NAME = conf_settings.vdb_index


# Environment Variables

In [65]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

# Clients

In [66]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model=LLM_WORKHORSE,
    )
try:
    _ = llm.invoke("tell me a joke about devops")
except Exception as err:
    print(err)

    
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model=EMBEDDINGS)
try:
    _ = embeddings.embed_query("healthcheck")

except Exception as err:
    print(err)



vector_store = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name=INDEX_NAME,
    url=VDB_URL,
    api_key=QDRANT_API_KEY,
)
try:
    _ = vector_store.asimilarity_search("healthcheck")
except Exception as err:
    print(err)




C:\Users\manua\AppData\Local\Temp\ipykernel_2412\2808977317.py:6: RuntimeWarning: coroutine 'VectorStore.asimilarity_search' was never awaited
  _ = llm.invoke("tell me a joke about devops")


In [67]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model=LLM_WORKHORSE,
    )



# Cleaning

In [68]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from pydantic import BaseModel, Field
from enum import Enum

    


qrewrite_template = """You are an assistant tasked with taking a natural languge query from a user
    and converting it into a query for a vectorstore. In the process, strip out all 
    infohrmation that is not relevant for te retrieval task and return a new, simplified
    question for vectorstore retrieval.
    For example:
    * Fix typing mistakes
    * Remove irrelevant information for the retrieval task
    Question: {question} """

prompt_qrewrite= ChatPromptTemplate.from_template(qrewrite_template)
chain_qrewrite = prompt_qrewrite | llm

chain_qrewrite.invoke(
    {
        "question": "I love nature. What are the primary causes of the decline in bee populations globally?",
    }
)


AIMessage(content='What are the primary causes of the decline in bee populations globally?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 111, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CF7OBv93pOn23NHXQGnNVmXRHgbOp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--18cf9ab4-26a7-41ac-a0d9-3446f02d52bc-0', usage_metadata={'input_tokens': 111, 'output_tokens': 13, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [69]:
q_raw1 = "Que telescopio descubrio Althera. Di si fue un telescopio optico o infrarrojo o de otra clase"

q_ref1 = chain_qrewrite.invoke({"question": q_raw1})
q_ref1

AIMessage(content='Qué telescopio descubrió Althera y de qué tipo era (óptico, infrarrojo u otro).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 119, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CF7ODYdXFXdvz3nZyuZj6EbzM6vNM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--15bb98bd-303d-46d8-a63b-2cf46805e4cf-0', usage_metadata={'input_tokens': 119, 'output_tokens': 24, 'total_tokens': 143, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [70]:
q_raw2 = "Quien descubrió Althera, la NASA o la ESA"

q_ref2 = chain_qrewrite.invoke({"question": q_raw2})
q_ref2

AIMessage(content='Who discovered Althera?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 107, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CF7OE57x2hsAKHXpeEqsX791F1a1m', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--95707c0a-2217-4182-915b-0623ddd61b84-0', usage_metadata={'input_tokens': 107, 'output_tokens': 6, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [71]:
from rag import main as rag_agent_builder

rag_graph = rag_agent_builder("space", 3)


resp = rag_graph.invoke({"question": q_raw1})
resp['answer']

C:\Users\manua\AppData\Local\Temp\ipykernel_2412\2193883185.py:3: RuntimeWarning: coroutine 'VectorStore.asimilarity_search' was never awaited
  rag_graph = rag_agent_builder("space", 3)


'El telescopio que descubrió el sistema binario Althéra fue el Observatorio Espacial James Webb. Este es un telescopio principalmente de infrarrojo.'

In [72]:
resp = rag_graph.invoke({"question": q_ref1.content})
resp['answer']

'El sistema binario Althéra fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb. Este observatorio es un telescopio espacial de tipo infrarrojo.'

# Follow up

In [73]:
from typing import Optional  
from pydantic import BaseModel, Field


class Contextualizer(BaseModel):
    """ 
    Determine whether a user query is properly contextualized for retrieval.
    """
    is_contextualized: bool = Field(..., description="True if the query is properly contextualized, False otherwise")
    query_examples: Optional[str]  = Field(..., description="If not properly contextualized, provide a list of similar questions that you consider properly contextualized")
    


qfollowup_template = """You are an assistant tasked with taking a natural languge query from a user
    and evaluating whether it is properly contextualized to perform a retrieval task.
    -----
    Examples:
    Question: ¿Que seguro es más barato?
    is_contextualized: False
    query_examples: [¿Que seguro de coche es más barato?, ¿Que seguro de autos es más barato?]
    ------
    Question: {question} """

prompt_qfollowup= ChatPromptTemplate.from_template(qfollowup_template)
chain_qfollowup = prompt_qfollowup | llm.with_structured_output(Contextualizer )

chain_qfollowup.invoke(
    {
        "question": "function string to datetime",
    }
)


Contextualizer(is_contextualized=False, query_examples='How to convert string to datetime in Python?; Function to parse string into datetime object; String to datetime conversion function in JavaScript.')

# MultiQuery Retrieval


![multiqueryrewrite.png](docs/multiqueryrewrite.png)

In [74]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "¿Qué observatorio confirmó la existencia de cinco planetas principales?"
llm = ChatOpenAI(temperature=0.4)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(),
    llm=llm,
    include_original=True
)

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

retriever_from_llm.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. ¿Cuál institución científica verificó la presencia de cinco planetas principales?', '2. ¿Qué centro de investigación validó la existencia de cinco planetas principales?', '3. ¿Qué organización astronómica confirmó la presencia de cinco planetas principales?']


[Document(metadata={'Header 1': '1. Historia del descubrimiento', 'Header 2': '1.2 Confirmación mediante técnicas combinadas', '_id': 'ef1949a3-3b45-4907-9927-9072a28aad83', '_collection_name': 'space'}, page_content='## 1.2 Confirmación mediante técnicas combinadas  \nEn los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  \n- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.\n- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.\n- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  \nFue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de esc

In [75]:
question = "¿Qué instrumento confirmó la existencia de cinco planetas principales?"

retriever_from_llm.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. ¿Cuál fue el instrumento utilizado para verificar la presencia de cinco planetas principales?', '2. ¿Qué dispositivo se utilizó para confirmar la existencia de cinco planetas principales?', '3. ¿Qué herramienta científica se empleó para corroborar la presencia de cinco planetas principales?']


[Document(metadata={'Header 1': '1. Historia del descubrimiento', 'Header 2': '1.2 Confirmación mediante técnicas combinadas', '_id': 'ef1949a3-3b45-4907-9927-9072a28aad83', '_collection_name': 'space'}, page_content='## 1.2 Confirmación mediante técnicas combinadas  \nEn los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  \n- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.\n- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.\n- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  \nFue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de esc

In [76]:
question = "DAG en Airflow"

retriever_from_llm.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the relationship between Directed Acyclic Graphs (DAGs) and Airflow?', '2. How does Airflow utilize Directed Acyclic Graphs (DAGs) in its workflow management system?', '3. Can you explain the integration of DAGs and Airflow in data pipeline orchestration?']


[Document(metadata={'Header 1': '2. Conoce a Althéra', 'Header 2': '2.2 Disposición y dinámica de un sistema binario', '_id': 'd4bf1036-0a1c-4683-bdfe-b1d663f5ef6a', '_collection_name': 'space'}, page_content='## 2.2 Disposición y dinámica de un sistema binario'),
 Document(metadata={'Header 1': '8. Fenómenos destacados', '_id': 'c0b37e44-9b92-475a-981d-c9f0054e2b50', '_collection_name': 'space'}, page_content="# 8. Fenómenos destacados  \nEl sistema Althéra es un escenario dinámico donde los fenómenos astronómicos se manifiestan con una belleza y complejidad difícil de encontrar en sistemas de una sola estrella. Entre ellos, uno de los más notables es la interacción constante entre el Cinturón de Arges y los planetas interiores, que da lugar a lluvias de meteoros dobles visibles desde Aurelia III. Debido a la iluminación combinada de Althéra A y B, estos meteoros generan estelas de colores inusuales: plateadas bajo la luz dorada de Althéra A y anaranjadas bajo la luz más cálida de Alt

```python 

# Default prompt
DEFAULT_QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is
    to generate 3 different versions of the given user
    question to retrieve relevant documents from a vector  database.
    By generating multiple perspectives on the user question,
    your goal is to help the user overcome some of the limitations
    of distance-based similarity search. Provide these alternative
    questions separated by newlines. Original question: {question}""",
)


```


https://python.langchain.com/api_reference/_modules/langchain/retrievers/multi_query.html#MultiQueryRetriever

In [77]:
qexpa_template = """You are an assistant tasked with taking a natural languge query from a user
    and generating more suitable candidates to perform a retrieval task.
    Generate 3 different versions withouth changing the main goal of the question
    Focus on:
    * looking synonyms, related terms or adding explanations of key concepts in the query
    * Resolve well known acronyms
    * Translate terms to the original query language
    Provide these alternative questions separated by newlines. 
    -----
    Examples:
    Question: Is Python a good programming language?
    query_examples: [Is Python a performant programming language?, Does Python manages memory efficiently?, Is Python a good programming language for data science?]
    ------
    Original question: {question} """

# a good example
# https://smith.langchain.com/hub/smithing-gold/question-decomposition

prompt_qexpa = ChatPromptTemplate.from_template(qexpa_template)


question = "¿Qué observatorio confirmó la existencia de cinco planetas principales?"
retriever_from_llm_custom = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(),
    llm=llm,
    include_original=True,
    prompt=prompt_qexpa

)

retriever_from_llm_custom.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['query_examples: [¿Qué observatorio validó la existencia de cinco planetas principales?, ¿Qué institución confirmó la presencia de cinco planetas principales?, ¿Qué entidad verificó la existencia de cinco planetas principales?]']


[Document(metadata={'Header 1': '1. Historia del descubrimiento', 'Header 2': '1.2 Confirmación mediante técnicas combinadas', '_id': 'ef1949a3-3b45-4907-9927-9072a28aad83', '_collection_name': 'space'}, page_content='## 1.2 Confirmación mediante técnicas combinadas  \nEn los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  \n- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.\n- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.\n- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  \nFue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de esc

In [78]:
question = "¿Qué instrumento confirmó la existencia de cinco planetas principales?"
retriever_from_llm_custom.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['Alternative questions:', '¿Qué instrumento validó la existencia de cinco planetas principales?', '¿Qué instrumento demostró la existencia de cinco planetas principales?', '¿Qué instrumento corroboró la existencia de cinco planetas principales?']


[Document(metadata={'Header 1': '6. Planetas exteriores', 'Header 2': 'Adéndum: ¿Son las lunas exteriores superhabitables?', '_id': '9f16ad07-b5b0-4021-b771-0a848e8f5b25', '_collection_name': 'space'}, page_content='## Adéndum: ¿Son las lunas exteriores superhabitables?  \nSi bien los planetas gigantes exteriores no son habitables, varias de sus lunas ofrecen escenarios tentadores para la astrobiología. Selvaris , con su océano interno protegido por hielo, podría albergar ecosistemas impulsados por calor geotérmico y reacciones químicas entre agua y minerales del núcleo. Calther , a pesar de su superficie gélida y órbita inestable, podría contener bolsas internas de amoníaco líquido que actúen como anticongelante natural, manteniendo microambientes habitables durante parte de su órbita.'),
 Document(metadata={'Header 1': '7. Lunas y satélites menores', 'Header 2': 'Adéndum: ¿Y si la vida se esconde?', '_id': '142249ea-030d-4e27-9fdd-40bb544c5c12', '_collection_name': 'space'}, page_con

In [79]:
question = "¿Fue la NASA quein descubrió Althera?"
retriever_from_llm_custom.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['query_examples: [¿Fue la NASA la organización que descubrió Althera?, ¿Fue la NASA responsable del descubrimiento de Althera?, ¿Fue la NASA quien encontró Althera?]']


[Document(metadata={'Header 1': '1. Historia del descubrimiento', 'Header 2': '1.4 Importancia científica y proyección futura', '_id': 'dea169aa-22d8-42e7-b722-fe80677ccf90', '_collection_name': 'space'}, page_content='## 1.4 Importancia científica y proyección futura  \nEl descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:  \n1. Dinámica circumbinaria estable - demostró que los planetas pueden mantener órbitas estables y climas equilibrados alrededor de dos soles, desafiando modelos anteriores.\n2. Química atmosférica compleja - Aurelia III presenta una mezcla de gases que, en equilibrio fotoquímico, sugieren procesos biológicos o geoquímicos activos.\n3. Proximidad relativa - su cercanía permite observaciones directas en la próxima década con telescopios como el Habitable Worlds Observatory (HWO) y misiones de espectrometría directa de superficie como Starshot Spectra .  \nEn 2036, la Unión Astronómica Internacional otorgó a este hallaz

In [80]:
question = "¿Fue la ESA quein descubrió Althera?"
retriever_from_llm_custom.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['query_examples: [¿Fue la Agencia Espacial Europea la que descubrió Althera?, ¿Quién fue responsable del descubrimiento de Althera por parte de la ESA?, ¿La ESA fue la entidad que encontró Althera?]']


[Document(metadata={'Header 1': '1. Historia del descubrimiento', 'Header 2': '1.4 Importancia científica y proyección futura', '_id': 'dea169aa-22d8-42e7-b722-fe80677ccf90', '_collection_name': 'space'}, page_content='## 1.4 Importancia científica y proyección futura  \nEl descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:  \n1. Dinámica circumbinaria estable - demostró que los planetas pueden mantener órbitas estables y climas equilibrados alrededor de dos soles, desafiando modelos anteriores.\n2. Química atmosférica compleja - Aurelia III presenta una mezcla de gases que, en equilibrio fotoquímico, sugieren procesos biológicos o geoquímicos activos.\n3. Proximidad relativa - su cercanía permite observaciones directas en la próxima década con telescopios como el Habitable Worlds Observatory (HWO) y misiones de espectrometría directa de superficie como Starshot Spectra .  \nEn 2036, la Unión Astronómica Internacional otorgó a este hallaz